In [ ]:
!pip install mcpl
!pip install scipy
!pip install mpi4py
!pip install trame
!pip install ipywidgets

# Prompt V1.0

In [ ]:
!prompt -g box.gdml -n 100 

# Plot, not verbose

In [ ]:
from cinema_plot import tutorial_run
tutorial_run('cubes.gdml', numNeu=100, showInJupyter=True)

Prompt V2.0 feature

In [ ]:
from Cinema.Prompt import Prompt, PromptMPI
from Cinema.Prompt.geo import Volume, Transformation3D
from Cinema.Prompt.solid import Box
from Cinema.Prompt.gun import PythonGun
import numpy as np

class MySim(Prompt):
    def __init__(self, seed=4096) -> None:
        super().__init__(seed)

    def makeWorld(self, anyUserParameters=np.zeros(2)):
        scorerCfg_detpsd = "Scorer=PSD;name=NeutronHistMap;xmin=-500;xmax=500;numbin_x=10;ymin=-500;ymax=500;numbin_y=10;ptstate=SURFACE;type=XY"
        scorerCfg_detwl = "Scorer=WlSpectrum; name=detector; min=0.0; max=5; numbin=200"
        matCfg_sample = "physics=ncrystal;nccfg='LiquidHeavyWaterD2O_T293.6K.ncmat';scatter_bias=10;abs_bias=1"

        world = Volume("world", Box(210, 210, 210))
        detector = Volume("Det", Box(400, 400, 0.0001))
        detector.addScorer(scorerCfg_detpsd)
        detector.addScorer(scorerCfg_detwl)

        sample = Volume("sample", Box(20,20,20))
        water = Volume('sbox', Box(2,2,2), matCfg = matCfg_sample)

        for i in range(10):
            sample.placeChild("ssample", water, Transformation3D(0,0,2.2*(i-5.0)), scorerGroup=i)

        world.placeChild("physicalbox", detector, Transformation3D(0., 0., 50), 1)
        world.placeChild("physicalbox2", sample, Transformation3D(-150, 45, 45))
        self.l.setWorld(world)


class MyGun(PythonGun):
    def __init__(self):
        super().__init__()

    def samplePosition(self):
        return 0,0,-209


sim = MySim(seed=1010)
sim.makeWorld()

# set gun
gun = MyGun()
sim.setGun(gun)
sim.simulate(1000, True, True)